# "Anticipez les besoins en consommation électrique de bâtiments"
_Notebook_

## 0 Preliminaries

### 0.0 Importing Packages and Modules

Checking whether the notebook is on Colab or PC

In [1]:
import sys
is_colab = 'google.colab' in sys.modules
is_colab, sys.executable

(True, '/usr/bin/python3')

Mounting my Drive if on colab

In [2]:
if is_colab==True:
    from google.colab import files, output, drive
    drive.mount('/gdrive')
    %cd /gdrive
    print("You're on Google Colab")
else:
    print("You're on a PC")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
You're on Google Colab


Noticing user about files that has to be in the current working directory.

In [3]:
def file_required(file_name):
    if os.path.exists(os.getcwd()+"/"+file_name):
        print("OK, loading "+file_name+"\
                           from the current working directory ...")
    else:
        print("Please, put "+file_name+"\
                           in the current working directory")

Installations and importations required for Google Colab version.

In [4]:
# !pip freeze > requirements.txt

In [ ]:
import os

if is_colab==True:
    if os.getcwd()!='/gdrive/My Drive/--DATA SCIENCE/PROJET4':
        os.chdir('My Drive/--DATA SCIENCE/PROJET4')
    !pip install -r requirements.txt
else:
    # make sure 'P4_functions.py' is in the current working directory
    file_required('/P4_functions.py')
    file_required('requirements.txt')
    !pip install -r requirements.txt

     |████████████████████████████████| 133kB 4.2MB/s 
     |████████████████████████████████| 6.3MB 6.7MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 163kB 52.3MB/s 
     |████████████████████████████████| 880kB 52.0MB/s 
     |████████████████████████████████| 2.0MB 61.4MB/s 
     |████████████████████████████████| 7.5MB 46.7MB/s 
     |████████████████████████████████| 163kB 53.9MB/s 
     |████████████████████████████████| 3.0MB 58.9MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 368kB 51.5MB/s 
     |████████████████████████████████| 29.8MB 148kB/s 
     |████████████████████████████████| 1.1MB 47.5MB/s 
     |████████████████████████████████| 307kB 49.2MB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
     |████████████████████████████████| 1.3MB 57.0MB/s 
     |████████████████████████████████| 1.3MB 54.1MB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
  

In [ ]:
from P4_functions import *

Importation of modules and packages. 

In [ ]:
import io
from time import time

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor']='w'

import warnings
warnings.filterwarnings('ignore')

# from ipywidgets import interact, interact_manual

import dill

from sklearn_pandas import DataFrameMapper

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mlxtend.plotting import plot_pca_correlation_graph
from sklearn.inspection import permutation_importance

import category_encoders as ce

import scipy.stats as st

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import *
from sklearn import metrics
from sklearn.compose import make_column_transformer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR

Setting pandas display options

In [ ]:
dictPdSettings = {'display.max_rows': 500, 'display.width': 100,
                    'display.max_colwidth': 500,
                    'display.float_format': lambda x: '%.2f' % x}
for k,v in dictPdSettings.items():
    pd.set_option(k,v)

To play audio text-to-speech during execution.

In [ ]:
from IPython.display import Audio
from gtts import gTTS

def speak(text, lang='en'):
    with io.BytesIO() as f:
        gTTS(text=text, lang=lang).write_to_fp(f)
        f.seek(0)
        return Audio(f.read(), autoplay=True)

In [ ]:
plot_enabled = False
save_enabled = False

In [ ]:
speak('Packages and modules successfully imported')

### 0.1 Importing the dataset

In [ ]:
if is_colab==True:
    path_csv = 'DATA/Cleaned_data.csv' 
    # Importing database from my Drive
    print("2. Try to import in notebook from myDrive...")
else:
    path_csv = '../DATA/Cleaned_data.csv'
    # Importing database from a DATA directory
    print("2. Try to import in notebook from PC ('DATA')")

df = pd.read_csv(path_csv, sep=',', index_col='Unnamed: 0',
                 low_memory=True, encoding ='utf-8')

print("-----> importation of .csv in the notebook OK")

In [ ]:
df['ZipCode'] = df['ZipCode'].apply(str).astype('object')
df['CouncilDistrictCode'] = df['CouncilDistrictCode'].apply(str).astype('object')

In [ ]:
speak('Dataset successfully imported')

### 0.2 Overview

In [ ]:
df.shape

In [ ]:
df.columns

Printing total nb and percentage of null

In [ ]:
print_null_pct(df)

Showing columns where there are still NaN values

In [ ]:
df.isna().sum(axis=0)[df.isna().sum(axis=0)!=0]

In [ ]:
speak('Description overview done')

## 1 Data preprocessing

### 1.1 Target data transformation

We already found in the Cleaning_EDA notebook that the distribution of the target data could become almost normal with logarithmic transformation.

Here are the histograms of transformed values.

#### SiteEnergyUseWN(kBtu)

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].hist(df['SiteEnergyUseWN(kBtu)'], color='grey', ec='k',
            histtype='stepfilled', bins=50);
axs[0].set_xlabel("'SiteEnergyUseWN(kBtu)'")
axs[1].hist(df['SiteEnergyUseWN(kBtu)_log'], color='grey', ec='k',
            histtype='stepfilled', bins=50);
axs[1].set_xlabel("log of 'SiteEnergyUseWN(kBtu)'")


plt.tight_layout(rect=[0,0,1,0.95])
fig.suptitle('SiteEnergyUseWN(kBtu)', fontweight='bold')
fig.set_size_inches(7,3)
plt.show()

#### TotalGHGEmissions

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].hist(df['TotalGHGEmissions'], color='grey', ec='k',
            histtype='stepfilled', bins=50);
axs[0].set_xlabel("'TotalGHGEmissions'")
axs[1].hist(df['TotalGHGEmissions_log'], color='grey', ec='k',
            histtype='stepfilled', bins=50);
axs[1].set_xlabel("log of 'TotalGHGEmissions'")


plt.tight_layout(rect=[0,0,1,0.95])
fig.suptitle('TotalGHGEmissions', fontweight='bold')
fig.set_size_inches(7,3)
plt.show()

### 1.2 Features selection

In [ ]:
prop_Q_cols = ['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
               'PropertyGFAParking', 'PropertyGFABuilding(s)',
               'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA',
               'ThirdLargestPropertyUseTypeGFA', 'BuildingAge',
               'MeanGFAperFloor', 'ParkingGFARatio', 'NbYearsCertified',
               'ExtsurfVolRatio',
            'Outlier'] # 'ENERGYSTARScore',
prop_C_cols = ['Neighborhood', 'ZipCode', 'CouncilDistrictCode',
               'BuildingType', 'PrimaryPropertyType', 'LargestPropertyUseType',
               'SecondLargestPropertyUseType', 'ThirdLargestPropertyUseType',
               'CertifiedPreviousYear', 'EnergyProfile', 'MainEnergy']
target_cols = ['SiteEnergyUseWN(kBtu)', 'TotalGHGEmissions',
                'SiteEnergyUseWN(kBtu)_log', 'TotalGHGEmissions_log']

__Quantitative data__

Excluding highly correlated quantitative features.
- 'PropertyGFATotal' is strongly correlated with 'PropertyGFABuilding(s)'
- 'PropertyGFABuilding(s)' is strongly correlated with 'LargestPropertyUseTypeGFA'
- 'ExtsurfVolRatio' is strongly correlated with 'NumberofBuildings'

In [ ]:
# prop_Q_cols_sel = [c for c in prop_Q_cols if c not in\
#                               ['PropertyGFATotal',
#                                'PropertyGFABuilding(s)',
#                                'ExtsurfVolRatio']]

__Categorical data__

Excluding highly related categorical features.
- 'LargestPropertyUseType'(57) is strongly correlated with 'PrimaryPropertyType'(29)

In [ ]:
# prop_C_cols_sel = [c for c in prop_C_cols if c not in ['LargestPropertyUseType']]

### 1.3 Splitting training and testing set 

In [ ]:
df_X = df[prop_Q_cols+prop_C_cols]
df_y1 = df['SiteEnergyUseWN(kBtu)']
df_y2 = df['TotalGHGEmissions']

We want to evaluate our model at the end on various 'BuildingType'.

We thus split the data with stratification using this column.

In [ ]:
''' 'Neighborhood', 'CouncilDistrictCode', 'ZipCode', 'Outlier', 'BuildingType',
'PrimaryPropertyType', 'EnergyProfile', 'MainEnergy', 'CertifiedPreviousYear' '''

group_col = 'PrimaryPropertyType'

X_tr, X_te, y1_tr, y1_te = train_test_split(df_X, df_y1,
                                            train_size=0.8, shuffle=True,
                                            stratify=df[group_col],
                                            random_state=14)

In [ ]:
y1_log_tr, y1_log_te  = np.log(y1_tr), np.log(y1_te)

Printing the quantitative and categorical independent columns:

In [ ]:
print(f'--- {X_tr.columns.shape[0]} independent columns ---\
\nquantitative ({len(prop_Q_cols)}): {prop_Q_cols},\
\ncategorical ({len(prop_C_cols)}): {prop_C_cols}')

In [ ]:
# print("--- columns missing in prop_Q_cols_sel and prop_C_cols_sel ---\n" ,
#       set(X_tr.columns) - set(X_tr_sel.columns))

### 1.4 Feature transformations

We want to evaluate the impact of the transformation strategies for quantitative and categorical data. We'll split the columns in 4 types.

__Quantitative data__

Integers or float.

__Categorical data__

Categorical data are all nominal data, none of them are ordinal.
- Boolean : 'CertifiedPreviousYear'
- low cardinality (less than 12 categories):
'BuildingType', 'MainEnergy', 'EnergyProfile', 'Outlier'
- high cardinality (12 or more categories) :
('Neighborhood', 'LargestPropertyUseType',
 'SecondLargestPropertyUseType',
 'ThirdLargestPropertyUseType', 'PrimaryPropertyType')

The transformation of the columns is all included in the Class CustTransformer (see P4_funtions.py file), and the different encoding and transformation options are passed to the GridSearchCV or RandomizedSearchCV throudh the param_grid dictionnary.

## 2 First model selection

In [ ]:
#  # TESTS RAPIDES
# my_knn = KNeighborsRegressor(n_neighbors=5)

# cust_trans = CustTransformer(thresh_card=12,
#                              strat_binary='ord',
#                              strat_low_card='ohe',
#                              strat_high_card='hash',
#                              strat_quant='stand').fit(X_tr, y1_tr)
# X_tr_enc = cust_trans.transform(X_tr)
# my_knn.fit(X_tr_enc, y1_tr)
# X_te_enc = cust_trans.transform(X_te)
# y1_te_pr = my_knn.predict(X_te_enc)

# scores_reg('my_knn', X_te_enc, y1_te, y1_te_pr)

In [ ]:
# If file of models exists, open and load in dict_model
if os.path.exists(os.getcwd()+'/P4_models.pkl'):
    file = open(os.getcwd()+'/P4_models.pkl', 'rb')
    dict_models = dill.load(file)
    print('Pickle containing models already existing:\n', dict_models.keys())
    print("Content loaded in 'dict_models'")
# Else create an empty dictionnary
else:
    dict_models = {}
    print('No pickle containing models.')
    print("Created an empty 'dict_models' ditionnary")

In [ ]:
df_res = pd.DataFrame(dtype = 'object')

### 2.0 Dummy Regressor baseline

In [ ]:
from sklearn.dummy import DummyRegressor

reg = DummyRegressor()
name_reg = 'Dummy'

param_grid={'preproc__strat_binary': ['ord'],
            'preproc__strat_low_card': ['bin'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_high_card': ['bin'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_quant': ['stand'],  #'stand', 'minmax', 'maxabs', 'robust', 'norm', 'quant_uni', 'quant_norm', 'pow'
            name_reg+'__strategy': ['mean']} # 'mean', 'median'

# If model with the name already in the pickle, just take the loaded model
if dict_models.get(name_reg, np.nan) is not np.nan:
    print('-----Model already exists - taking loaded model from pickle')
# Else computes the new model and add to the dictionnary, and then to the pickle
else:
    print('-----Model does not exist in the pickle - computing...')
    t0 = time()
    # refit is set by default with 'neg_root_mean_squared_error'
    dict_models[name_reg] = \
        model_optimizer(CustTransformer(thresh_card=12),
                        name_reg, reg, param_grid,
                        X_tr, y1_tr,
                        cv_search=StratifiedKFold(5).split(X_tr, X_tr[group_col]),
                        scv_scores = ['neg_root_mean_squared_error', 
                                      'neg_mean_absolute_error',
                                      'r2'],
                        search_strat='grid')#, n_iter=50)
    print("---> Optimization took {:0.3f}s.".format(time()-t0))
    file = open(os.getcwd()+'/P4_models.pkl', 'wb')
    try:  # PAS TROUVE D'AUTRE SOLUTION POUR EVITER LE MESSAGE D'ERREUR
        dill.dump(dict_models, file)
    except:
        print("-----...model dumped")

new_df_res = scv_perf_fetcher(name_reg, dict_models[name_reg])
df_res = pd.concat([df_res, new_df_res], axis=1)

### 2.1 KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

reg = KNeighborsRegressor()
name_reg = 'KNN_rand'
param_grid={'preproc__strat_binary': ['ord'],
            'preproc__strat_low_card': ['bin', 'ord'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_high_card': ['bin', 'ord', 'hash'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_quant': ['stand'], # 'stand', 'minmax', 'maxabs', 'robust', 'norm', 'quant_uni', 'quant_norm', 'pow'
            name_reg+'__n_neighbors': [1,2,3,5,7,9,11,15,17], # 1,2,3,5,7,9,11,15,17
            name_reg+'__p': [1,2]} # 0,1,2

# If model with the name already in the pickle, just take the loaded model
if dict_models.get(name_reg, np.nan) is not np.nan:
    print('-----Model already exists - taking loaded model from pickle')
# Else computes the new model and add to the dictionnary, and then to the pickle
else:
    print('-----Model does not exist in the pickle - computing...')
    t0 = time()
    # refit is set by default with 'neg_root_mean_squared_error'
    dict_models[name_reg] = \
        model_optimizer(CustTransformer(thresh_card=12),
                        name_reg, reg, param_grid,
                        X_tr, y1_tr,
                        cv_search=StratifiedKFold(5).split(X_tr, X_tr[group_col]),
                        scv_scores = ['neg_root_mean_squared_error', 
                                      'neg_mean_absolute_error',
                                      'r2'],
                        search_strat='rand', n_iter=200)
    print("---> Optimization took {:0.3f}.".format(time()-t0))
    file = open(os.getcwd()+'/P4_models.pkl', 'wb')
    try:  # PAS TROUVE D'AUTRE SOLUTION POUR EVITER LE MESSAGE D'ERREUR
        dill.dump(dict_models, file)
    except:
        print("-----...model dumped")

new_df_res = scv_perf_fetcher(name_reg, dict_models[name_reg])
df_res = pd.concat([df_res, new_df_res], axis=1)

### 2.2 Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

reg = Ridge()
name_reg = 'Ridge_rand'
param_grid={'preproc__strat_binary': ['ord'],
            'preproc__strat_low_card': ['ohe', 'bin'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_high_card': ['ohe', 'bin'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_quant': ['stand'], # 'stand', 'minmax', 'maxabs', 'robust', 'norm', 'quant_uni', 'quant_norm', 'pow'
            name_reg+'__alpha': st.loguniform(1e-7, 1e7)}
            # name_reg+'__alpha': np.logspace(-7,7,20)}

# If model with the name name already in the pickle, just take the loaded model
if dict_models.get(name_reg, np.nan) is not np.nan:
    print('-----Model already exists - taking loaded model from pickle')
# Else computes the new model and add to the dictionnary, and then to the pickle
else:
    print('-----Model does not exist in the pickle - computing...')
    t0 = time()
    # refit is set by default with 'neg_root_mean_squared_error'
    dict_models[name_reg] = \
        model_optimizer(CustTransformer(thresh_card=12),
                        name_reg, reg, param_grid,
                        X_tr, y1_tr,
                        cv_search=StratifiedKFold(5).split(X_tr, X_tr[group_col]),
                        scv_scores = ['neg_root_mean_squared_error', 
                                      'neg_mean_absolute_error',
                                      'r2'],
                        search_strat='rand', n_iter=200)
    print("---> Optimization took {:0.3f}s.".format(time()-t0))
    file = open(os.getcwd()+'/P4_models.pkl', 'wb')
    try:  # PAS TROUVE D'AUTRE SOLUTION POUR EVITER LE MESSAGE D'ERREUR
        dill.dump(dict_models, file)
    except:
        print("-----...model dumped")

new_df_res = scv_perf_fetcher(name_reg, dict_models[name_reg])
df_res = pd.concat([df_res, new_df_res], axis=1)

### 2.3 Lasso

In [ ]:
from sklearn.linear_model import Lasso

reg = Lasso()
name_reg = 'Lasso_rand'
param_grid={'preproc__strat_binary': ['ord'],
            'preproc__strat_low_card': ['ohe', 'bin'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_high_card': ['bin', 'ord', 'hash'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_quant': ['stand'], # 'stand', 'minmax', 'maxabs', 'robust', 'norm', 'quant_uni', 'quant_norm', 'pow'
            name_reg+'__alpha': st.loguniform(1e-7, 1e7)}
            # name_reg+'__alpha': np.logspace(-7,7,20)}

# If model with the name name already in the pickle, just take the loaded model
if dict_models.get(name_reg, np.nan) is not np.nan:
    print('-----Model already exists - taking loaded model from pickle')
# Else computes the new model and add to the dictionnary, and then to the pickle
else:
    print('-----Model does not exist in the pickle - computing...')
    t0 = time()
    # refit is set by default with 'neg_root_mean_squared_error'
    dict_models[name_reg] = \
        model_optimizer(CustTransformer(thresh_card=12),
                        name_reg, reg, param_grid,
                        X_tr, y1_tr,
                        cv_search=StratifiedKFold(5).split(X_tr, X_tr[group_col]),
                        scv_scores = ['neg_root_mean_squared_error', 
                                      'neg_mean_absolute_error',
                                      'r2'],
                        search_strat='rand', n_iter=200)
    print("---> Optimization took {:0.3f}s.".format(time()-t0))
    file = open(os.getcwd()+'/P4_models.pkl', 'wb')
    try:  # PAS TROUVE D'AUTRE SOLUTION POUR EVITER LE MESSAGE D'ERREUR
        dill.dump(dict_models, file)
    except:
        print("-----...model dumped")

new_df_res = scv_perf_fetcher(name_reg, dict_models[name_reg])
df_res = pd.concat([df_res, new_df_res], axis=1)

### 2.4 ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet

reg = ElasticNet()
name_reg = 'ElasticNet_rand'
param_grid={'preproc__strat_binary': ['ord'],
            'preproc__strat_low_card': ['ohe', 'bin', 'hash'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_high_card': ['ord', 'hash', 'loo'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_quant': ['stand'], # 'stand', 'minmax', 'maxabs', 'robust', 'norm', 'quant_uni', 'quant_norm', 'pow'
            name_reg+'__alpha': st.loguniform(1e-7, 1e7), # np.logspace(-7,7,20)
            name_reg+'__l1_ratio': st.uniform(0,1), # [0,0.15,0.3,0.45,0.6,0.75,1]
            name_reg+'__tol' : st.loguniform(1e-3, 1e0)} #np.logspace(-4,0,5)}

# If model with the name name already in the pickle, just take the loaded model
if dict_models.get(name_reg, np.nan) is not np.nan:
    print('-----Model already exists - taking loaded model from pickle')
# Else computes the new model and add to the dictionnary, and then to the pickle
else:
    print('-----Model does not exist in the pickle - computing...')
    t0 = time()
    # refit is set by default with 'neg_root_mean_squared_error'
    dict_models[name_reg] = \
        model_optimizer(CustTransformer(thresh_card=12),
                        name_reg, reg, param_grid,
                        X_tr, y1_tr,
                        cv_search=StratifiedKFold(5).split(X_tr, X_tr[group_col]),
                        scv_scores = ['neg_root_mean_squared_error', 
                                      'neg_mean_absolute_error',
                                      'r2'],
                        search_strat='rand', n_iter=200)
    print("---> Optimization took {:0.3f}s.".format(time()-t0))
    file = open(os.getcwd()+'/P4_models.pkl', 'wb')
    try:  # PAS TROUVE D'AUTRE SOLUTION POUR EVITER LE MESSAGE D'ERREUR
        dill.dump(dict_models, file)
    except:
        print("-----...model dumped")

new_df_res = scv_perf_fetcher(name_reg, dict_models[name_reg])
df_res = pd.concat([df_res, new_df_res], axis=1)

### 2.5 Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# {'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 500}

reg = RandomForestRegressor()
name_reg = 'RandomForest_grid'
param_grid={'preproc__strat_binary': ['ord'],
            'preproc__strat_low_card': ['ohe', 'bin', 'hash'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_high_card': ['ord', 'hash', 'loo'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_quant': ['stand'], # 'stand', 'minmax', 'maxabs', 'robust', 'norm', 'quant_uni', 'quant_norm', 'pow'
            name_reg+'__n_estimators': [100,200,500], # nb leaves
            name_reg+'__min_samples_leaf': [1,2,3,5]} # mean nb leaves in a node
            name_reg+'__max_features': ['auto', 'sqrt']} # 'auto', 'sqrt' (nb features in a tree)

# If model with the name name already in the pickle, just take the loaded model
if dict_models.get(name_reg, np.nan) is not np.nan:
    print('-----Model already exists - taking loaded model from pickle')
# Else computes the new model and add to the dictionnary, and then to the pickle
else:
    print('-----Model does not exist in the pickle - computing...')
    t0 = time()
    # refit is set by default with 'neg_root_mean_squared_error'
    dict_models[name_reg] = \
        model_optimizer(CustTransformer(thresh_card=12),
                        name_reg, reg, param_grid,
                        X_tr, y1_tr,
                        cv_search=StratifiedKFold(5).split(X_tr, X_tr[group_col]),
                        scv_scores = ['neg_root_mean_squared_error', 
                                      'neg_mean_absolute_error',
                                      'r2'],
                        search_strat='grid', verbose=2)#, n_iter=5)
    print("---> Optimization took {:0.3f}s.".format(time()-t0))
    file = open(os.getcwd()+'/P4_models.pkl', 'wb')
    try:  # PAS TROUVE D'AUTRE SOLUTION POUR EVITER LE MESSAGE D'ERREUR
        dill.dump(dict_models, file)
    except:
        print("-----...model dumped")

new_df_res = scv_perf_fetcher(name_reg, dict_models[name_reg])
df_res = pd.concat([df_res, new_df_res], axis=1)

### 2.6 Support Vector Machine Regression

In [ ]:
from sklearn.svm import SVR

reg = SVR()
name_reg = 'SVR_grid'
param_grid={'preproc__strat_binary': ['ord'],
            'preproc__strat_low_card': ['ohe', 'bin', 'hash'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_high_card': ['ord', 'hash', 'loo'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_quant': ['stand'], # 'stand', 'minmax', 'maxabs', 'robust', 'norm', 'quant_uni', 'quant_norm', 'pow'
            name_reg+'__gamma': np.logspace(-8,0,9), # kernel coef (radial basis function 'rbf')
            name_reg+'__epsilon': np.logspace(-3,0,4), # accepted error
            name_reg+'__C': np.logspace(-2,2,5)} # regularization coef

# If model with the name name already in the pickle, just take the loaded model
if dict_models.get(name_reg, np.nan) is not np.nan:
    print('-----Model already exists - taking loaded model from pickle')
# Else computes the new model and add to the dictionnary, and then to the pickle
else:
    print('-----Model does not exist in the pickle - computing...')
    t0 = time()
    # refit is set by default with 'neg_root_mean_squared_error'
    dict_models[name_reg] = \
        model_optimizer(CustTransformer(thresh_card=12),
                        name_reg, reg, param_grid,
                        X_tr, y1_tr,
                        cv_search=StratifiedKFold(5).split(X_tr, X_tr[group_col]),
                        scv_scores = ['neg_root_mean_squared_error', 
                                      'neg_mean_absolute_error',
                                      'r2'],
                        search_strat='grid')#, n_iter=5)
    print("---> Optimization took {:0.3f}s.".format(time()-t0))
    file = open(os.getcwd()+'/P4_models.pkl', 'wb')
    try:  # PAS TROUVE D'AUTRE SOLUTION POUR EVITER LE MESSAGE D'ERREUR
        dill.dump(dict_models, file)
    except:
        print("-----...model dumped")

new_df_res = scv_perf_fetcher(name_reg, dict_models[name_reg])
df_res = pd.concat([df_res, new_df_res], axis=1)

### 2.7 XGBoost Regressor

In [ ]:
from xgboost import XGBRegressor

reg = XGBRegressor()
name_reg = 'XGBR_grid'
param_grid={'preproc__strat_binary': ['ord'],
            'preproc__strat_low_card': ['ohe', 'bin', 'hash'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_high_card': ['ord', 'hash', 'loo'], # 'ohe', 'bin', 'ord', 'hash', 'loo'
            'preproc__strat_quant': ['stand'], # 'stand', 'minmax', 'maxabs', 'robust', 'norm', 'quant_uni', 'quant_norm', 'pow'
            # name_reg+'__n_estimators': np.logspace(-8,-1,8), # kernel coef (radial basis function 'rbf')
            name_reg+'__n_estimators': [100,500,1000,2000]} # regularization coef

# If model with the name name already in the pickle, just take the loaded model
if dict_models.get(name_reg, np.nan) is not np.nan:
    print('-----Model already exists - taking loaded model from pickle')
# Else computes the new model and add to the dictionnary, and then to the pickle
else:
    print('-----Model does not exist in the pickle - computing...')
    t0 = time()
    # refit is set by default with 'neg_root_mean_squared_error'
    dict_models[name_reg] = \
        model_optimizer(CustTransformer(thresh_card=12),
                        name_reg, reg, param_grid,
                        X_tr, y1_tr,
                        cv_search=StratifiedKFold(5).split(X_tr, X_tr[group_col]),
                        scv_scores = ['neg_root_mean_squared_error', 
                                      'neg_mean_absolute_error',
                                      'r2'],
                        search_strat='grid')#, n_iter=5)
    print("---> Optimization took {:0.3f}s.".format(time()-t0))
    file = open(os.getcwd()+'/P4_models.pkl', 'wb')
    try:  # PAS TROUVE D'AUTRE SOLUTION POUR EVITER LE MESSAGE D'ERREUR
        dill.dump(dict_models, file)
    except:
        print("-----...model dumped")

new_df_res = scv_perf_fetcher(name_reg, dict_models[name_reg])
df_res = pd.concat([df_res, new_df_res], axis=1)

In [ ]:
speak("youpi cé fini")

## 3 Comparison of the models
Plotting the main metrics of the optimized models obtained during SearchCV fitting (training/test folds)

In [ ]:
df_res

### Learning curves

In [ ]:
## LENT MAIS FONCTIONNE, prévoir différents types de lignes

# fig = None
# colors = sns.color_palette("dark", 10)
# %matplotlib
# for name, color in zip(dict_models.keys(), colors):
#     print(name,'...')
#     fig = plot_learning_curve(dict_models[name].best_estimator_, X_tr, y1_tr,
#                             train_sizes=np.linspace(0.1,1,30),
#                             scoring="neg_mean_squared_error", label=name,
#                             c=color, fig=fig) # to add the curve to existing figure
#     print('... -> done !')
    
# %matplotlib inline
# fig